In [18]:
%env ENERGY_PLUS_FOLDER=/Applications/EnergyPlus-8-5-0/
%env DEMO_FILE_FOLDER=/Applications/EnergyPlus-8-5-0/ExampleFiles/BasicsFiles/
%env COMMON_IDD_FILE=/Applications/EnergyPlus-8-5-0/Energy+.idd
%env DEMO_FILE=Exercise1A.idf
%env EPLUS2FMU_SCRIPT_FOLDER=/Users/Trondle/Downloads/EnergyPlusToFMU-v2/Scripts

env: ENERGY_PLUS_FOLDER=/Applications/EnergyPlus-8-5-0/
env: DEMO_FILE_FOLDER=/Applications/EnergyPlus-8-5-0/ExampleFiles/BasicsFiles/
env: COMMON_IDD_FILE=/Applications/EnergyPlus-8-5-0/Energy+.idd
env: DEMO_FILE=Exercise1A.idf
env: EPLUS2FMU_SCRIPT_FOLDER=/Users/Trondle/Downloads/EnergyPlusToFMU-v2/Scripts


## Run Using EnergyPlus

In [11]:
rm -rf ./temp

In [12]:
%%bash
mkdir temp
cd temp
cp $DEMO_FILE_ORIGIN/$DEMO_FILE .
energyplus $DEMO_FILE

EnergyPlus Starting
EnergyPlus, Version 8.5.0-c87e61b44b, YMD=2016.08.09 10:33
Processing Data Dictionary
Processing Input File
Initializing Simulation
Reporting Surfaces
Beginning Primary Simulation
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
Warming up {6}
Starting Simulation at 07/21 for CHICAGO_IL_USA COOLING .4% CONDITIONS DB=>MWB
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
Warming up {6}
Starting Simulation at 01/21 for CHICAGO_IL_USA HEATING 99.6% CONDITIONS
Writing final SQL reports
EnergyPlus Run Time=00hr 00min  0.28sec


## Create FMU File

In [24]:
rm -rf ./temp

In [25]:
%%bash
mkdir temp
cd temp
cp $DEMO_FILE_ORIGIN/$DEMO_FILE .
cat <<EOT >> $DEMO_FILE
  ExternalInterface,
    FunctionalMockupUnitExport;  !- Name of External Interface

  ExternalInterface:FunctionalMockupUnitExport:From:Variable,
    Environment, !- EnergyPlus Key Value
    Zone Air Temperature, !- EnergyPlus Variable Name
    AirTemp; !- FMU Variable Name
EOT
python $EPLUS2FMU_SCRIPT_FOLDER/EnergyPlusToFMU.py -i $COMMON_IDD_FILE $DEMO_FILE

Reading input and weather file for EnergyPlusToFMU program.


clang: warning: argument unused during compilation: '-fast'
/Users/Trondle/Downloads/EnergyPlusToFMU-v2/SourceCode/EnergyPlus/temp-Exercise1A.c:1377:3: warning: '&&' within '||' [-Wlogical-op-parentheses]
                && _c->curComm <=(_c->tStopFMU - 
                ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/Users/Trondle/Downloads/EnergyPlusToFMU-v2/SourceCode/EnergyPlus/temp-Exercise1A.c:1377:3: note: place parentheses around the '&&' expression to silence this warning
                && _c->curComm <=(_c->tStopFMU - 
                ^
1 warning generated.
clang: warning: argument unused during compilation: '-fast'
clang: warning: argument unused during compilation: '-fast'
clang: warning: argument unused during compilation: '-fast'
clang: warning: argument unused during compilation: '-fast'
/Users/Trondle/Downloads/EnergyPlusToFMU-v2/SourceCode/EnergyPlus/xml_parser_cosim.c:427:13: warning: comparison of constant -1 with expression of type 'Elm' is always false [-Wtautological-constant-o

In [27]:
ls -l temp

total 1992
-rw-r--r--  1 Trondle  staff  643084  9 Aug 11:19 Exercise1A.fmu
-rw-r--r--  1 Trondle  staff   20311  9 Aug 11:19 Exercise1A.idf
-rwxr-xr-x  1 Trondle  staff  351108  9 Aug 11:19 idf-to-fmu-export-prep-darwin*


## Run FMU Co-Simulation

In [34]:
import os
import pyfmi
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
%matplotlib inline

In [39]:
DEMO_FILE = (Path('.') / 'temp' / os.environ['DEMO_FILE']).with_suffix('.fmu')
T_START = 0
T_END = 2 * 24 * 4
NUMBER_TIME_STEPS = 2 * 24 * 4
T_STEP_SIZE = float(T_END) / NUMBER_TIME_STEPS
TIME_STEPS = np.linspace(T_START, T_END, NUMBER_TIME_STEPS)

In [40]:
model = pyfmi.load_fmu(DEMO_FILE.as_posix())
model.instantiate_slave()
model.initialize(T_START, T_END, True)

FMUException: PyFMI does not support co-simulation tool

Apparently FMU co-simulation is not always of the same type. There seem to be two different types: one in which the solver is included in the compiled FMU, and one in which the FMU calls another tool (on the command line). PyFMI does not support the latter. EnergyPlus2FMU uses the latter. Hence, PyFMI and EnergyPlus2FMU currently do not work together. [Others have ran into the same probem before and managed to solve it.](https://groups.google.com/forum/#!topic/energyplus-fmu/d9AoiLYoGtk)

The main developer of PyFMI [stated at the end of 2015](http://www.jmodelica.org/27626) that this won't be supported officially antime soon.